In [5]:
import os
import glob
import math
import time
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import warnings
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import random
import seaborn as sns; sns.set_theme()
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import pylab as pl
from matplotlib.pyplot import figure
from IPython import display
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn import svm
from numpy import std
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import cm
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [8]:
train = pd.read_csv('./MLR_Project_train.csv')
test = pd.read_csv('./MLR_Project_test.csv')

## 1.3 Show the maximum return of train and test

In [9]:
train_max = np.sum(train['TARGET'][train['TARGET']>0])
test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Maximum return of training set:', train_max)
print('Maximum return of testing set:', test_max)

Maximum return of training set: 195.6927566509
Maximum return of testing set: 55.96225182400002


### 1.3.1 Remove the Unnamed columns in dataframe

In [10]:
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]

In [11]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,TARGET
0,5874.524387,1072.671848,41440.76212,41440.23732,70.405148,7.392780,70.377281,23229.69262,23229.72655,70.378864,7.389173,70.380160,23229.76782,23379.81637,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.425618,70.494241,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,4898.757333,15165.927590,297487.1654,297487.16540,15165.927590,4898.757333,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,64.192982,20.940618,76.070270,23376.73707,23343.13291,77.290965,347.308164,102.380501,24823.08137,24120.94894,332.757607,17.386711,129.622187,23936.99077,21670.192330,71.518948,11.399004,78.006816,26437.161240,23811.09670,141.997532,22.474794,0.013314
1,6124.154099,1072.802927,41440.76212,41442.22458,70.456758,7.356050,70.379576,23229.76020,23230.10472,70.273618,7.389813,70.329906,23229.46908,23384.98219,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.441223,70.702624,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5100.350569,15788.076830,308790.4312,308790.43120,15788.076830,5100.350569,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,2.937218,-14.428210,75.204973,23317.46049,23309.42032,74.880368,689.670872,127.070357,36746.26762,31012.79786,-310.576721,4.532673,-47.045260,22053.23653,14626.733390,48.124991,-99.618253,-115.120518,7705.543821,22665.35143,-377.287072,-73.700375,-0.000448
2,5905.732593,1072.802927,41443.14358,41442.30403,70.422472,7.417794,70.376448,23229.48142,23229.61008,70.474265,7.388979,70.397301,23229.83396,23387.39168,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.440201,70.686178,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5308.548086,16430.607950,320463.9968,320463.99680,16430.607950,5308.548086,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,141.231442,25.855392,75.184756,23248.26780,23307.91084,74.635569,634.523047,71.705965,28917.00549,24632.17456,419.071308,7.403187,118.846496,23430.24573,31251.552920,71.535567,53.482719,106.179152,37586.677270,23251.62576,261.098973,22.565621,0.000244
3,6029.325221,1072.802927,41440.76212,41442.25682,70.458533,7.366031,70.379221,23230.08433,23229.87971,70.288944,7.390120,70.370247,23229.81662,23390.02296,83.418623,11.615135,83.418623,23233.34325,23466.72590,83.418623,7.439191,70.692085,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5510.265781,17053.141280,331774.2409,75710.89648,17053.141280,5510.265781,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,-1.580478,-7.499746,75.184756,23248.26836,23307.91084,74.635569,583.001082,70.384022,25642.94219,23482.26269,-249.671869,7.388974,49.809681,23193.67720,15867.015790,70.369496,-12.169114,63.930236,10052.351290,23229.64352,-10.549985,4.656636,-0.000628
4,6142.360146,1072.802927,41443.14358,41442.46480,70.413623,7.411287,70.376788,23229.70975,23229.82255,70.467206,7.389910,70.386986,23229.87603,23392.66710,70.572881,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.438244,70.680386,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,1152.667997,17699.014450,343508.5253,343508.52530,17699.014450,5719.546213,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,149.144800,26.789196,74.904988,23229.77108,23260.32245,70.506980,566.549008,120.694073,25765.82131,24618.79392,363.188022,7.389057,78.826922,23235.86490,31790.064250,120.694068,42.971377,145.572170,37109.895810,24143.94971,188.639704,31.863254,0.003811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [12]:
train_ = (train.iloc[:, :-1]-train.iloc[:, :-1].mean())/train.iloc[:, :-1].std()
train_['TARGET'] = train['TARGET']
train = train_

test_ = (test.iloc[:, :-1]-test.iloc[:, :-1].mean())/test.iloc[:, :-1].std()
test_['TARGET'] = test['TARGET']
test = test_

In [13]:
train['group'] = 1

quantiles = [train['TARGET'].quantile(x/10) for x in range(1, 10)]

for i in range(train.shape[0]):
    for j in range(9):
        if train['TARGET'][i] < quantiles[j]:
            train['group'][i] = j
            break
        train['group'][i] = 9

In [14]:
test['group'] = 1

# quantiles = [test['TARGET'].quantile(x/10) for x in range(1, 10)]

for i in range(test.shape[0]):
    for j in range(9):
        if test['TARGET'][i] < quantiles[j]:
            test['group'][i] = j
            break
        test['group'][i] = 9

In [16]:
train = train.sort_values(by=['TARGET'])
test = test.sort_values(by=['TARGET'])

In [17]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,TARGET,group
5311,-1.828848,0.061590,0.051644,0.069584,1.966939,-0.521111,0.585660,0.351703,1.096275,-0.272259,0.758817,-0.160207,0.208601,-0.230611,-0.334985,-0.479180,-0.360251,-0.429031,-0.584708,-0.480010,0.482215,-0.213232,0.279682,-0.157947,0.331524,0.537145,-0.630849,0.004700,-0.480997,-0.649021,-0.177134,-0.232891,-0.110614,-0.499240,-0.687222,-0.564201,-0.702255,-0.488955,1.807739,0.870800,-0.848010,0.484384,-0.343316,-0.742311,0.596984,-0.134172,-0.490265,0.527276,-0.282894,0.205439,-0.066223,0.067349,0.145583,0.246842,-0.442351,0.254895,-0.095598,0.065827,-0.982258,-0.339882,-1.151386,-0.902825,-0.735209,0.107738,-0.501095,-0.264170,-0.154823,0
86042,-2.195944,-1.131669,-1.121669,-1.130584,-0.243144,-1.289818,-0.731321,-0.759621,-0.993678,-0.027374,-1.205494,-0.871645,-0.971125,-0.230613,-0.334985,-0.505720,-0.390189,-0.366388,-0.531599,-0.420148,0.329011,1.170822,-0.630875,0.004978,0.737510,0.537145,1.188150,-0.484165,0.331131,-0.056281,-0.365545,-0.552776,-0.429854,-0.527037,-0.715209,-0.592136,-0.802336,0.281819,0.619311,1.808739,1.953859,-0.873104,-0.542159,-0.841302,0.645710,-0.015194,-0.102165,0.443747,-0.192722,0.094515,1.008998,0.178016,0.435217,0.398264,0.357679,-0.204605,-0.074520,-0.210228,0.842611,0.312083,0.397236,0.378051,0.614088,-0.073499,0.085584,-0.041586,-0.142635,0
74611,-1.903237,-1.648748,-1.658610,-1.644295,1.074970,-0.372081,5.418552,0.047033,2.100590,1.276948,2.557401,-0.458479,0.124363,-0.230615,-0.303654,-0.134164,-0.270437,2.640486,2.017674,2.453253,0.970955,2.145809,0.279682,1.960081,2.361454,2.612105,0.278650,-0.810075,-0.480997,-0.649021,-0.458215,0.359819,-0.313897,2.250044,2.080792,2.198682,-0.822600,-0.017884,1.445894,1.355372,-0.360259,-0.920159,-0.963400,-1.051012,0.908699,0.054547,0.281063,-0.174997,-0.250673,-0.286961,2.286478,0.692805,1.432326,1.055583,0.884669,0.925669,0.993718,0.927555,-0.633774,-0.048233,-0.479706,-0.526806,0.417129,0.783521,0.630521,0.586739,-0.142378,0
71753,-1.807234,-1.867512,-1.877363,-1.859787,1.880033,-1.316230,-0.776351,0.120794,-0.222184,-1.235688,-0.773798,-0.901094,-0.631289,-0.230616,-0.366316,2.015551,-0.390189,-0.397709,-0.531599,-0.450079,-0.141115,1.030117,-0.630875,-0.972573,-0.886435,-0.944970,1.188150,0.167655,0.331131,-0.056281,1.717617,2.073947,1.831015,-0.556565,-0.522671,-0.621810,-0.862456,-0.928404,-1.038510,-1.088099,1.044940,0.579901,0.825797,-0.160281,-0.710609,-0.740737,-0.328561,0.300690,-0.456636,-0.248759,1.724299,1.317544,1.208813,1.033882,1.495607,1.043426,-0.427519,0.216219,-0.909574,-2.333380,-2.350349,-2.388337,0.770809,0.129476,-1.211593,-0.663773,-0.112402,0
41790,0.109154,0.280355,0.290284,0.277596,0.690201,-0.684487,0.013742,0.878054,0.716381,0.357663,0.257792,-0.271271,0.187762,-0.022440,-0.397646,-0.479180,-0.360251,-0.241101,-0.425379,-0.300423,0.713489,1.207522,0.279682,-0.157947,0.737510,0.537145,1.188150,-0.647119,-0.074932,-0.352651,-0.493941,-0.360145,-0.237611,2.199916,2.030323,2.148306,-0.533872,-0.590093,0.971538,0.419965,0.653724,-0.903230,-0.811850,-0.975565,0.870825,-1.135328,0.669663,-0.599072,0.203221,-0.183247,0.875933,-0.430692,-0.055167,-0.352431,2.135527,0.006137,0.371192,0.144629,1.095594,0.035028,-0.167186,-0.114493,2.071204,0.016885,0.239938,0.080056,-0.103030,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40318,-1.616596,-0.017960,-0.008016,-0.027430,2.349345,-2.393817,-2.940176,-1.649764,-2.273420,-2.049505,-3.129286,-2.532108,-2.589916,-0.230607,2.641430,2.015551,2.453915,-0.366388,-0.531599,-0.420148,-0.632090,0.668038,0.279682,-0.809648,-0.480449,-0.648547,2.097650,0.004700,0.33

In [51]:
# reg = Ridge(alpha=0.5).fit(pd.DataFrame(sample.iloc[:, :66]), sample['TARGET'])
sample = train.iloc[45000:49100, :]
reg = LinearRegression().fit(pd.DataFrame(sample.iloc[:, :66]), sample['TARGET'])
train_pred = reg.predict(pd.DataFrame(sample.iloc[:, :66]))
figure(figsize=(8, 6), dpi=80)
plt.scatter(sample['TARGET'], train_pred, c = 'blue')
plt.show()

# reg = Ridge(alpha=0.5).fit(pd.DataFrame(sample.iloc[:, :66]), sample['TARGET'])
sample = test.iloc[10000:11000, :]
reg = LinearRegression().fit(pd.DataFrame(sample.iloc[:, :66]), sample['TARGET'])
test_pred = reg.predict(pd.DataFrame(sample.iloc[:, :66]))
figure(figsize=(8, 6), dpi=80)
plt.scatter(sample['TARGET'], test_pred, c = 'blue')
plt.show()

In [54]:
from sklearn.metrics.pairwise import euclidean_distances
dist = euclidean_distances(test.iloc[:, :-2].to_numpy(), test.iloc[:, :-2].to_numpy())

In [61]:
figure(figsize=(8, 8), dpi=80)
plt.imshow((dist-np.min(dist))/(np.max(dist)-np.min(dist)))
plt.colorbar()
plt.show()

In [68]:
plt.hist((dist-np.min(dist))/(np.max(dist)-np.min(dist)))
plt.show()

In [77]:
pca = PCA(n_components=20)
pca.fit(train.iloc[:, :-2])
train_var = pca.explained_variance_ratio_
train_pca = pca.fit_transform(train.iloc[:, :-2])
# reducer = umap.UMAP()
# train_umap = reducer.fit_transform(train_std_noncor.iloc[:, :-1])
# print(train_umap.shape)

In [78]:
pca = PCA(n_components=20)
pca.fit(test.iloc[:, :-2])
test_var = pca.explained_variance_ratio_
test_pca = pca.fit_transform(test.iloc[:, :-2])
# reducer = umap.UMAP()
# train_umap = reducer.fit_transform(train_std_noncor.iloc[:, :-1])

In [80]:
(test_var-train_var)/train_var

array([ 0.00595512,  0.071406  ,  0.00139604, -0.04950684,  0.01667597,
       -0.04795553,  0.03148047,  0.07795067,  0.00757195,  0.01357144,
       -0.03801875,  0.00844005, -0.04730374,  0.01469014, -0.00855045,
        0.04254564, -0.02787654, -0.00336187, -0.09609555, -0.04324409])

In [83]:
train_std = (train.iloc[:, :-2] - train.iloc[:, :-2].mean())/train.iloc[:, :-2].std()
test_std = (test.iloc[:, :-2] - test.iloc[:, :-2].mean())/test.iloc[:, :-2].std()

In [84]:
pca = PCA(n_components=20)
pca.fit(train_std.iloc[:, :])
train_std_var = pca.explained_variance_ratio_
train_std_pca = pca.fit_transform(train.iloc[:, :])
# reducer = umap.UMAP()
# train_umap = reducer.fit_transform(train_std_noncor.iloc[:, :-1])
# print(train_umap.shape)

In [85]:
pca = PCA(n_components=20)
pca.fit(test_std.iloc[:, :])
test_std_var = pca.explained_variance_ratio_
test_std_pca = pca.fit_transform(test.iloc[:, :])
# reducer = umap.UMAP()
# train_umap = reducer.fit_transform(train_std_noncor.iloc[:, :-1])

In [86]:
(test_std_var-train_std_var)/train_std_var

array([ 0.00595512,  0.071406  ,  0.00139604, -0.04950684,  0.01667597,
       -0.04795553,  0.03148047,  0.07795067,  0.007572  ,  0.01357145,
       -0.03801903,  0.00843986, -0.0473039 ,  0.01468694, -0.00855319,
        0.04253553, -0.02796717, -0.00300291, -0.09651884, -0.04326188])